In [1]:
from sklearn.datasets import make_classification
import pandas as pd 
import numpy as np 

In [2]:
X, y = make_classification(n_samples=400, n_features=4)
X = pd.DataFrame(X, columns=['f1', 'f2', 'f3', 'f4'])
y = pd.Series(y)

In [3]:
y

0      1
1      0
2      0
3      1
4      1
      ..
395    0
396    1
397    1
398    0
399    1
Length: 400, dtype: int32

In [91]:
def manual_roc_auc(y_true, y_scores):
    # 1. Разделим все объекты на положительные (label == 1) и отрицательные (label == 0)
    positives = [(score, label) for score, label in zip(y_scores, y_true) if label == 1]
    negatives = [(score, label) for score, label in zip(y_scores, y_true) if label == 0]

    P = len(positives)  # Количество положительных примеров
    N = len(negatives)  # Количество отрицательных примеров

    # 2. Проверка: если одного из классов нет, метрика не считается
    if P == 0 or N == 0:
        return None  # Нельзя вычислить AUC без положительных или отрицательных примеров

    total = 0  # Накопитель для суммы сравнения скоров

    # 3. Проходим по всем отрицательным примерам
    for neg_score, _ in negatives:
        # Считаем, сколько положительных примеров имеют больший скор
        count_higher = sum(1 for pos_score, _ in positives if pos_score > neg_score)
        # Считаем, сколько положительных примеров имеют такой же скор
        count_equal = sum(1 for pos_score, _ in positives if pos_score == neg_score)
        # Добавляем к сумме: полное количество выше + половину от равных
        total += count_higher + 0.5 * count_equal

    # 4. Нормализация суммы: делим на общее количество пар (P * N)
    auc = total / (P * N)
    return auc

y_scores = [0.91, 0.86, 0.78, 0.6, 0.6, 0.55, 0.51, 0.46, 0.45, 0.45, 0.42]
y_true =   [1,    0,    0,   1,   0,   1,   0,   0,   0,   1,   0]

result = manual_roc_auc(y_true, y_scores)
print(f"Ручной ROC AUC: {result:.5f}")


Ручной ROC AUC: 0.60714


In [77]:
class MyLogReg():
    def __init__(self, n_iter=10, learning_rate=0.1, metric=None, weights=None):
        self.n_iter = n_iter 
        self.learning_rate = learning_rate
        self.__weights = weights
        self.metric = metric
        self.__best_score = None

        self.__validate_params()
        
    def __repr__(self):
        return f'MyLogReg class: n_iter={self.n_iter}, learning_rate={self.learning_rate}'

    def __validate_params(self):
        """Проверяет корректность параметров модели."""
        if self.metric is not None and self.metric not in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']:
            raise ValueError(f"Invalid metric: {self.metric}, You can only use: 'accuracy', 'precision', 'recall', 'f1' or 'roc_auc' ")
    
    def __log_training_step(self, iteration: int, X: np.array, y: np.array, proba: np.array):
        """Логирует процесс обучения (итерацию, веса, функцию потерь).
    
        Параметры:
        iteration: int - номер итерации
        X: np.array - матрица признаков
        y: np.array - вектор истинных меток (0 или 1)
        proba: np.array - предсказанные вероятности класса 1
        """
        eps = 1e-15
        loss = -np.mean(y*np.log(proba+eps) + (1-y)*(np.log(1-proba+eps)))
        
        metric_value = None                                                 # Инициализируем метрикику 
        if self.metric:                                                     # если она задана 
            y_pred = (proba>0.5).astype(int)                                # вероятности переводим в метки
            metric_value = self.__calculat_metric(y, y_pred if self.metric != 'roc_auc' else proba)       # cчитаем метрику
            
        if iteration == 0:
            if self.metric:
                print(f'start | loss: {loss:0.2f} | {self.metric}: {metric_value:0.2f}')
            else:
                print(f'start | loss: {loss:0.2f}')
        else:
            if self.metric:
                print(f'{iteration} | loss: {loss:0.2f} | {self.metric}: {metric_value:0.2f}')
            else:
                print(f'{iteration} | loss: {loss:0.2f}')

    def __calculat_metric(self, y_true: np.array, y_pred: np.array):
        """
        Считает заданные метрики
        """
        #  Напишем функцию для подсчета матрицы ошибок
        def confusion_matrix(y_true: np.array, y_pred: np.array):
            # инициализируем нашу матрицу ошибок
            tn = fn = fp = tp = 0 
            for true, pred in zip(y_true, y_pred):
                if pred == 0 and true == 0:          # TN
                    tn += 1 
                elif pred == 0 and true == 1:        # FN
                    fn += 1 
                elif pred == 1 and true == 0:        # FP
                    fp += 1 
                elif pred == 1 and true == 1:        # TP
                    tp += 1 
            return tn, fn, fp, tp

        if self.metric == 'roc_auc':
            # Предсказанные вероятности
            y_scores = y_pred
            # Сортируем по вероятностям
            sorted_indices = np.argsort(y_scores)
            sorted_y = np.array(y_true)[sorted_indices]

            # Присваиваем ранги (от 1 до n, как в scipy)
            n = len(y_scores)
            ranks = np.empty(n)
            i = 0
            while i < n:
                j = i
                while j + 1 < n and y_scores[sorted_indices[j]] == y_scores[sorted_indices[j + 1]]:
                    j += 1
                avg_rank = (i + j + 2) / 2  # т.к. ранги начинаются с 1
                for k in range(i, j + 1):
                    ranks[k] = avg_rank
                i = j + 1

            # Считаем сумму рангов положительного класса
            sum_ranks_pos = np.sum(ranks[sorted_y == 1])

            P = np.sum(sorted_y)         # количество положительных
            N = n - P                    # количество отрицательных

            if P == 0 or N == 0:
                return None
            # Вычисляем AUC по формуле Манна-Уитни
            auc = (sum_ranks_pos - P * (P + 1) / 2) / (P * N)
            return auc

        tn, fn, fp, tp = confusion_matrix(y_true, y_pred)

        if self.metric == 'accuracy':
            return (tp + tn) / (tp + tn + fp + fn)
        elif self.metric == 'precision':
            return tp / (tp + fp)
        elif self.metric == 'recall':
            return tp / (tp + fn)
        elif self.metric == 'f1':
            pr = tp / (tp + fp)      # precision
            re = tp / (tp + fn)      # recall
            return (2*pr*re) / (pr + re)

    def fit(self, X: pd.DataFrame, y: pd.Series, verbose=False):
        """
        Метод обучает модель Логистической регресси 
        Входные параметры:
        X: pd.DataFrame
        y: pd.Series
        verbose: bool
        """
        X_copy = X.copy()                                     # копируем матрицу признаков, чтобы не изменить оригинальный 
        X_copy.insert(0, 'base', 1)                           # добавляем столбик для свободного члена, заполним его 1
        X_copy = X_copy.to_numpy()  
        self.__weights = np.ones(X_copy.shape[1])               # создаем вектор весов, заполненный 1

        # Цикл обучения
        for i in range(self.n_iter):
            pred = X_copy.dot(self.__weights)                    # делаем предсказание модели
            proba = 1 / (1 + np.exp(-pred))                    # переводим предсказания в вероятности через функцию сигмоиды
            grad = (1/len(y))*(proba - y).dot(X_copy)          # вычисляем градиент LogLoss
            #self.__weights -= self.learning_rate * grad          # делаем шаг обучения

            if self.metric:                                    # если задана метрика 
                if i == self.n_iter-1:                         # считаем его на n - 1 итерации, чтобы последнне значение записать в best_score
                    marks = (proba>0.5).astype(int)
                    self.__best_score = self.__calculat_metric(y_true=y, y_pred=marks if self.metric!='roc_auc' else proba)

            if verbose and (i % 10 == 0 or i == self.n_iter-1):
                self.__log_training_step(i, X_copy, y, proba)

            self.__weights -= self.learning_rate * grad          # делаем шаг обучения

    def predict_proba(self, X: pd.DataFrame) -> np.array:
        """Вернет вероятностное предсказание модели"""
        X_copy = X.copy()
        X_copy.insert(0, 'base', 1)
        pred = X_copy.dot(self.__weights)
        proba = 1 / (1 + np.exp(-pred))
        return np.array(proba)

    def predict(self, X: pd.DataFrame) -> np.array:
        """Вернет маркерное предсказание модели"""
        proba = self.predict_proba(X)
        return np.array((proba > 0.5).astype(int))

    def get_best_score(self):
        if self.metric is None:
            raise ValueError("Metric was not set during model initialization.")
        return self.__best_score
                
    def get_coef(self) -> np.array:
        return np.array(self.__weights[1:])


In [82]:
model = MyLogReg(n_iter=100, metric='roc_auc')

In [84]:
model.fit(X, y, verbose=True)

start | loss: 0.74 | roc_auc: 0.68
10 | loss: 0.49 | roc_auc: 0.91
20 | loss: 0.40 | roc_auc: 0.96
30 | loss: 0.35 | roc_auc: 0.97
40 | loss: 0.33 | roc_auc: 0.97
50 | loss: 0.31 | roc_auc: 0.97
60 | loss: 0.29 | roc_auc: 0.97
70 | loss: 0.28 | roc_auc: 0.97
80 | loss: 0.28 | roc_auc: 0.97
90 | loss: 0.27 | roc_auc: 0.97
99 | loss: 0.26 | roc_auc: 0.97


In [86]:
model.get_best_score()

0.9705242631065777

In [88]:
model.get_coef()

array([2.15576987, 0.16753171, 1.23971409, 0.63866576])

In [69]:
(model.predict_proba(X.iloc[:5]) > 0.5).astype(int)

array([1, 0, 0, 1, 1])

In [71]:
model.predict(X.iloc[:5])

array([1, 0, 0, 1, 1])